# **ETL**

In [146]:
#Importar las librerías necesarias
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import statsmodels.api as sm
from datetime import date
import seaborn as sns

In [147]:
#Opening the "Homicidios" file
homicidios = 'data/homicidios.xlsx'

## **"ETL Process for 'Hechos' Data"**

In [148]:
#Opening "Hechos" sheet
hechos = pd.read_excel(homicidios, sheet_name='HECHOS')

#Reviewing the data
hechos.head(3)

,ID,N_VICTIMAS,FECHA,AAAA,MM,DD,HORA,HH,LUGAR_DEL_HECHO,TIPO_DE_CALLE,...,Altura,Cruce,Dirección Normalizada,COMUNA,XY (CABA),pos x,pos y,PARTICIPANTES,VICTIMA,ACUSADO
0,2016-0001,1,2016-01-01,2016,1,1,04:00:00,4,AV PIEDRA BUENA Y AV FERNANDEZ DE LA CRUZ,AVENIDA,...,NaN,"FERNANDEZ DE LA CRUZ, F., GRAL. AV.","PIEDRA BUENA AV. y FERNANDEZ DE LA CRUZ, F., G...",8,Point (98896.78238426 93532.43437792),-58.47533969,-34.68757022,MOTO-AUTO,MOTO,AUTO
1,2016-0002,1,2016-01-02,2016,1,2,01:15:00,1,AV GRAL PAZ Y AV DE LOS CORRALES,GRAL PAZ,...,NaN,DE LOS CORRALES AV.,"PAZ, GRAL. AV. y DE LOS CORRALES AV.",9,Point (95832.05571093 95505.41641999),-58.50877521,-34.66977709,AUTO-PASAJEROS,AUTO,PASAJEROS
2,2016-0003,1,2016-01-03,2016,1,3,07:00:00,7,AV ENTRE RIOS 2034,AVENIDA,...,2034.0,NaN,ENTRE RIOS AV. 2034,1,Point (106684.29090040 99706.57687843),-58.39040293,-34.63189362,MOTO-AUTO,MOTO,AUTO


In [149]:
#Reviewing info
hechos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 696 entries, 0 to 695
Data columns (total 21 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   ID                     696 non-null    object        
 1   N_VICTIMAS             696 non-null    int64         
 2   FECHA                  696 non-null    datetime64[ns]
 3   AAAA                   696 non-null    int64         
 4   MM                     696 non-null    int64         
 5   DD                     696 non-null    int64         
 6   HORA                   696 non-null    object        
 7   HH                     696 non-null    object        
 8   LUGAR_DEL_HECHO        696 non-null    object        
 9   TIPO_DE_CALLE          696 non-null    object        
 10  Calle                  695 non-null    object        
 11  Altura                 129 non-null    float64       
 12  Cruce                  525 non-null    object        
 13  Direc

This general analysis shows that the dataset is fairly complete in terms of the data available for each incident, although some columns with null values might need more attention, such as ‘Altura’ and ‘Cruce’. However, these columns are not crucial for the analysis we are going to do. In conclusion, the various columns offer the opportunity to conduct a detailed analysis of the homicide incidents and to work on the KPIs.

In [150]:
#Let's see if there are duplicates
duplicados_hechos = hechos.duplicated().sum()
duplicados_hechos #There are no duplicate rows

0

In [151]:
#Vamos a ver si hay nulos
nulos_hechos = hechos.isnull().sum()
nulos_hechos #No hay nulos

ID                         0
N_VICTIMAS                 0
FECHA                      0
AAAA                       0
MM                         0
DD                         0
HORA                       0
HH                         0
LUGAR_DEL_HECHO            0
TIPO_DE_CALLE              0
Calle                      1
Altura                   567
Cruce                    171
Dirección Normalizada      8
COMUNA                     0
XY (CABA)                  0
pos x                      0
pos y                      0
PARTICIPANTES              0
VICTIMA                    0
ACUSADO                    0
dtype: int64

A possible reason for the high number of null values in height and crossing is that they are not required data. Since we already have the location information of the accidents from the addresses and coordinates, we can delete these data without losing much information.

In [152]:
#We are checking the null values in the standardized address
direccion_normalizada_nulos = hechos[hechos['Dirección Normalizada'].isnull()]
direccion_normalizada_nulos

,ID,N_VICTIMAS,FECHA,AAAA,MM,DD,HORA,HH,LUGAR_DEL_HECHO,TIPO_DE_CALLE,...,Altura,Cruce,Dirección Normalizada,COMUNA,XY (CABA),pos x,pos y,PARTICIPANTES,VICTIMA,ACUSADO
38,2016-0052,1,2016-04-20,2016,4,20,20:00:00,20,AUTOPISTA LUGONES PK 10000,AUTOPISTA,...,NaN,NaN,NaN,13,Point (. .),.,.,MOTO-SD,MOTO,SD
106,2016-0136,1,2016-10-25,2016,10,25,00:00:00,0,AU BUENOS AIRES - LA PLATA KM. 4,AUTOPISTA,...,NaN,NaN,NaN,4,Point (. .),.,.,MOTO-CARGAS,MOTO,CARGAS
119,2016-0151,1,2016-11-18,2016,11,18,20:35:00,20,SD,CALLE,...,NaN,NaN,NaN,0,Point (. .),.,.,PEATON-SD,PEATON,SD
180,2017-0050,2,2017-04-28,2017,4,28,11:08:08,11,AU PERITO MORENO Y RAMAL ENLACE AU1/AU6,AUTOPISTA,...,NaN,NaN,NaN,9,Point (. .),.,.,MOTO-CARGAS,MOTO,CARGAS
181,2017-0051,1,2017-05-01,2017,5,1,03:47:47,3,AU DELLEPIANE 2400,AUTOPISTA,...,NaN,NaN,NaN,7,Point (. .),.,.,AUTO-AUTO,AUTO,AUTO
313,2018-0039,1,2018-04-21,2018,4,21,22:15:00,22,AUTOPISTA LUGONES KM 4.7,AUTOPISTA,...,NaN,NaN,NaN,14,Point (. .),.,.,PEATON-AUTO,PEATON,AUTO
546,2020-0026,1,2020-05-17,2020,5,17,06:40:00,6,"LUGONES, LEOPOLDO AV. KM 6,1",AUTOPISTA,...,NaN,NaN,NaN,14,Point (. .),.,.,MOTO-OBJETO FIJO,MOTO,OBJETO FIJO
621,2021-0023,1,2021-03-01,2021,3,1,09:20:00,9,"AU BUENOS AIRES LA PLATA KM 4,5",AUTOPISTA,...,NaN,NaN,NaN,4,Point (. .),.,.,MOTO-CARGAS,MOTO,CARGAS


We can observe that in 88% of the cases, it is highway and only in one case it was in a place of fact ‘SD’, meaning no data. We will verify if there is a correlation between `Autopista` and normalized address `NaN`

In [153]:
#Filter cases where 'TIPO_DE_CALLE' is 'Autopista'
casos_autopista = hechos[hechos['TIPO_DE_CALLE'] == 'AUTOPISTA']

#Verify if 'Normalized Address' is always NaN.
es_direccion_nula_en_autopista = casos_autopista['Dirección Normalizada'].isnull().all()
es_direccion_nula_en_autopista

False

We see that there is no correlation, so we will avoid filling in those data, since it only accounts for 1% and will not affect our analysis.

In [154]:
#We are removing unnecessary columns.
hechos.drop(['Altura','Cruce'], axis=1, inplace=True)
hechos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 696 entries, 0 to 695
Data columns (total 19 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   ID                     696 non-null    object        
 1   N_VICTIMAS             696 non-null    int64         
 2   FECHA                  696 non-null    datetime64[ns]
 3   AAAA                   696 non-null    int64         
 4   MM                     696 non-null    int64         
 5   DD                     696 non-null    int64         
 6   HORA                   696 non-null    object        
 7   HH                     696 non-null    object        
 8   LUGAR_DEL_HECHO        696 non-null    object        
 9   TIPO_DE_CALLE          696 non-null    object        
 10  Calle                  695 non-null    object        
 11  Dirección Normalizada  688 non-null    object        
 12  COMUNA                 696 non-null    int64         
 13  XY (C

## **"ETL Process for 'VICTIMAS' Data"**

**Normalize the column names**

In [155]:
#We open the 'VICTIMAS' sheet in the 'homicidios' file for reading and transform it into a dataframe
victimas = pd.read_excel(homicidios, sheet_name='VICTIMAS')

victimas.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 717 entries, 0 to 716
Data columns (total 10 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   ID_hecho             717 non-null    object        
 1   FECHA                717 non-null    datetime64[ns]
 2   AAAA                 717 non-null    int64         
 3   MM                   717 non-null    int64         
 4   DD                   717 non-null    int64         
 5   ROL                  717 non-null    object        
 6   VICTIMA              717 non-null    object        
 7   SEXO                 717 non-null    object        
 8   EDAD                 717 non-null    object        
 9   FECHA_FALLECIMIENTO  717 non-null    object        
dtypes: datetime64[ns](1), int64(3), object(6)
memory usage: 56.1+ KB


In [156]:
#Let's see if there are duplicates
duplicados_victimas = victimas.duplicated().sum()
duplicados_victimas #No hay duplicados

0

In [157]:
#We are going to check for null values
nulos_victimas = victimas.isnull().sum()
nulos_victimas #There are no null values

ID_hecho               0
FECHA                  0
AAAA                   0
MM                     0
DD                     0
ROL                    0
VICTIMA                0
SEXO                   0
EDAD                   0
FECHA_FALLECIMIENTO    0
dtype: int64

In [158]:
#The variable 'ID_hecho' in 'victimas' is being renamed
victimas=victimas.rename(columns={'ID_hecho': 'ID'})
victimas.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 717 entries, 0 to 716
Data columns (total 10 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   ID                   717 non-null    object        
 1   FECHA                717 non-null    datetime64[ns]
 2   AAAA                 717 non-null    int64         
 3   MM                   717 non-null    int64         
 4   DD                   717 non-null    int64         
 5   ROL                  717 non-null    object        
 6   VICTIMA              717 non-null    object        
 7   SEXO                 717 non-null    object        
 8   EDAD                 717 non-null    object        
 9   FECHA_FALLECIMIENTO  717 non-null    object        
dtypes: datetime64[ns](1), int64(3), object(6)
memory usage: 56.1+ KB


In [159]:
#Now, we are merging both tables using a 'merge' operation
tabla_final = pd.merge(victimas, hechos, how='left', on='ID')
tabla_final.head()

,ID,FECHA_x,AAAA_x,MM_x,DD_x,ROL,VICTIMA_x,SEXO,EDAD,FECHA_FALLECIMIENTO,...,TIPO_DE_CALLE,Calle,Dirección Normalizada,COMUNA,XY (CABA),pos x,pos y,PARTICIPANTES,VICTIMA_y,ACUSADO
0,2016-0001,2016-01-01,2016,1,1,CONDUCTOR,MOTO,MASCULINO,19,2016-01-01 00:00:00,...,AVENIDA,PIEDRA BUENA AV.,"PIEDRA BUENA AV. y FERNANDEZ DE LA CRUZ, F., G...",8,Point (98896.78238426 93532.43437792),-58.47533969,-34.68757022,MOTO-AUTO,MOTO,AUTO
1,2016-0002,2016-01-02,2016,1,2,CONDUCTOR,AUTO,MASCULINO,70,2016-01-02 00:00:00,...,GRAL PAZ,"PAZ, GRAL. AV.","PAZ, GRAL. AV. y DE LOS CORRALES AV.",9,Point (95832.05571093 95505.41641999),-58.50877521,-34.66977709,AUTO-PASAJEROS,AUTO,PASAJEROS
2,2016-0003,2016-01-03,2016,1,3,CONDUCTOR,MOTO,MASCULINO,30,2016-01-03 00:00:00,...,AVENIDA,ENTRE RIOS AV.,ENTRE RIOS AV. 2034,1,Point (106684.29090040 99706.57687843),-58.39040293,-34.63189362,MOTO-AUTO,MOTO,AUTO
3,2016-0004,2016-01-10,2016,1,10,CONDUCTOR,MOTO,MASCULINO,18,SD,...,AVENIDA,LARRAZABAL AV.,"LARRAZABAL AV. y VILLEGAS, CONRADO, GRAL.",8,Point (99840.65224780 94269.16534422),-58.46503904,-34.68092974,MOTO-SD,MOTO,SD
4,2016-0005,2016-01-21,2016,1,21,CONDUCTOR,MOTO,MASCULINO,29,2016-02-01 00:00:00,...,AVENIDA,SAN JUAN AV.,"SAN JUAN AV. y SAENZ PEÃ‘A, LUIS, PRES.",1,Point (106980.32827929 100752.16915795),-58.38718297,-34.62246630,MOTO-PASAJEROS,MOTO,PASAJEROS


In [160]:
tabla_final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 717 entries, 0 to 716
Data columns (total 28 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   ID                     717 non-null    object        
 1   FECHA_x                717 non-null    datetime64[ns]
 2   AAAA_x                 717 non-null    int64         
 3   MM_x                   717 non-null    int64         
 4   DD_x                   717 non-null    int64         
 5   ROL                    717 non-null    object        
 6   VICTIMA_x              717 non-null    object        
 7   SEXO                   717 non-null    object        
 8   EDAD                   717 non-null    object        
 9   FECHA_FALLECIMIENTO    717 non-null    object        
 10  N_VICTIMAS             717 non-null    int64         
 11  FECHA_y                717 non-null    datetime64[ns]
 12  AAAA_y                 717 non-null    int64         
 13  MM_y 

Transform `FECHA_FALLECIMIENTO`

In [161]:
# Find rows where FECHA_FALLECIMIENTO does not match the expected datetime format
invalid_dates = tabla_final['FECHA_FALLECIMIENTO'].str.match(r'\d{4}-\d{2}-\d{2} \d{2}:\d{2}:\d{2}') == False

# Replace these values with NaN or some other placeholder
tabla_final.loc[invalid_dates, 'FECHA_FALLECIMIENTO'] = pd.NaT

# Now convert to datetime
tabla_final['FECHA_FALLECIMIENTO'] = pd.to_datetime(tabla_final['FECHA_FALLECIMIENTO'], errors='coerce')

# Extracting just the date part
tabla_final['FECHA_FALLECIMIENTO'] = tabla_final['FECHA_FALLECIMIENTO'].dt.date

Transform `HH`

In [162]:
# Replace "SD" with -1 to represent missing or undefined data
tabla_final['HH'].replace('SD', -1, inplace=True)

# Convert the "HH" column to integer type
tabla_final['HH'] = pd.to_numeric(tabla_final['HH'], errors='coerce', downcast='integer')

Transform 'pos x' and 'pos y'

In [163]:
# Filter no valid positions 
valid_positions = ~(tabla_final['pos x'].str.contains('[^0-9.,-]') | tabla_final['pos y'].str.contains('[^0-9.,-]') | (tabla_final['pos x'] == '.') | (tabla_final['pos y'] == '.'))

# Establecer posiciones no válidas en NaN
tabla_final.loc[~valid_positions, ['pos x', 'pos y']] = np.nan

# Convertir las cadenas a tipo numérico (float)
tabla_final['pos x'] = tabla_final['pos x'].str.replace(',', '.').astype(float)
tabla_final['pos y'] = tabla_final['pos y'].str.replace(',', '.').astype(float)

Transform `HORA`

In [164]:
# Replace 'SD' with '00:00:00' in the 'HORA' column
tabla_final['HORA'] = tabla_final['HORA'].replace('SD', '00:00:00')

# Convert 'HORA' to datetime format, keeping only the time part
# The errors='coerce' will handle any other unexpected formats and convert them to NaT
tabla_final['Hora'] = pd.to_datetime(tabla_final['HORA'], format='%H:%M:%S', errors='coerce')

# Remove any rows where conversion resulted in NaT
tabla_final = tabla_final.dropna(subset=['Hora'])

# Extract the time part in hh:mm:ss format
tabla_final['Hora'] = tabla_final['Hora'].dt.strftime('%H:%M:%S')

# Drop the original 'HORA' column as it's no longer needed
tabla_final.drop(columns=['HORA'], inplace=True)

# Now you can continue working with the updated DataFrame `htabla_final`
# For example, you can display the first few rows to check the changes
tabla_final.head()

,ID,FECHA_x,AAAA_x,MM_x,DD_x,ROL,VICTIMA_x,SEXO,EDAD,FECHA_FALLECIMIENTO,...,Calle,Dirección Normalizada,COMUNA,XY (CABA),pos x,pos y,PARTICIPANTES,VICTIMA_y,ACUSADO,Hora
0,2016-0001,2016-01-01,2016,1,1,CONDUCTOR,MOTO,MASCULINO,19,2016-01-01,...,PIEDRA BUENA AV.,"PIEDRA BUENA AV. y FERNANDEZ DE LA CRUZ, F., G...",8,Point (98896.78238426 93532.43437792),-58.475340,-34.687570,MOTO-AUTO,MOTO,AUTO,04:00:00
1,2016-0002,2016-01-02,2016,1,2,CONDUCTOR,AUTO,MASCULINO,70,2016-01-02,...,"PAZ, GRAL. AV.","PAZ, GRAL. AV. y DE LOS CORRALES AV.",9,Point (95832.05571093 95505.41641999),-58.508775,-34.669777,AUTO-PASAJEROS,AUTO,PASAJEROS,01:15:00
2,2016-0003,2016-01-03,2016,1,3,CONDUCTOR,MOTO,MASCULINO,30,2016-01-03,...,ENTRE RIOS AV.,ENTRE RIOS AV. 2034,1,Point (106684.29090040 99706.57687843),-58.390403,-34.631894,MOTO-AUTO,MOTO,AUTO,07:00:00
3,2016-0004,2016-01-10,2016,1,10,CONDUCTOR,MOTO,MASCULINO,18,NaT,...,LARRAZABAL AV.,"LARRAZABAL AV. y VILLEGAS, CONRADO, GRAL.",8,Point (99840.65224780 94269.16534422),-58.465039,-34.680930,MOTO-SD,MOTO,SD,00:00:00
4,2016-0005,2016-01-21,2016,1,21,CONDUCTOR,MOTO,MASCULINO,29,2016-02-01,...,SAN JUAN AV.,"SAN JUAN AV. y SAENZ PEÃ‘A, LUIS, PRES.",1,Point (106980.32827929 100752.16915795),-58.387183,-34.622466,MOTO-PASAJEROS,MOTO,PASAJEROS,05:20:00


In [165]:
tabla_final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 717 entries, 0 to 716
Data columns (total 28 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   ID                     717 non-null    object        
 1   FECHA_x                717 non-null    datetime64[ns]
 2   AAAA_x                 717 non-null    int64         
 3   MM_x                   717 non-null    int64         
 4   DD_x                   717 non-null    int64         
 5   ROL                    717 non-null    object        
 6   VICTIMA_x              717 non-null    object        
 7   SEXO                   717 non-null    object        
 8   EDAD                   717 non-null    object        
 9   FECHA_FALLECIMIENTO    648 non-null    object        
 10  N_VICTIMAS             717 non-null    int64         
 11  FECHA_y                717 non-null    datetime64[ns]
 12  AAAA_y                 717 non-null    int64         
 13  MM_y 

We can see that the columns `Fecha`, `Año`, `Mes`, `Día` y `Victima` are repeated, so we decide to remove them but before we are going to create a dataset for mySQL 

In [166]:
#Let's create a dataframe for SQL, with ID
SQL_homicidios=tabla_final.drop(['FECHA_y','AAAA_y', 'MM_y', 'DD_y', 'VICTIMA_y'], axis=1, inplace=True)

In [169]:
tabla_final.dtypes

ID                               object
FECHA_x                  datetime64[ns]
AAAA_x                            int64
MM_x                              int64
DD_x                              int64
ROL                              object
VICTIMA_x                        object
SEXO                             object
EDAD                             object
FECHA_FALLECIMIENTO              object
N_VICTIMAS                        int64
HH                                 int8
LUGAR_DEL_HECHO                  object
TIPO_DE_CALLE                    object
Calle                            object
Dirección Normalizada            object
COMUNA                            int64
XY (CABA)                        object
pos x                           float64
pos y                           float64
PARTICIPANTES                    object
ACUSADO                          object
Hora                             object
dtype: object

In [140]:
#Save dataset for MySQL
SQL_homicidios= tabla_final.to_csv("data/SQL_homicidios.csv", index=False)

In [141]:
tabla_final.drop(['ID'], axis=1, inplace=True)

In [142]:
tabla_final

,FECHA_x,AAAA_x,MM_x,DD_x,ROL,VICTIMA_x,SEXO,EDAD,FECHA_FALLECIMIENTO,N_VICTIMAS,...,TIPO_DE_CALLE,Calle,Dirección Normalizada,COMUNA,XY (CABA),pos x,pos y,PARTICIPANTES,ACUSADO,Hora
0,2016-01-01,2016,1,1,CONDUCTOR,MOTO,MASCULINO,19,2016-01-01,1,...,AVENIDA,PIEDRA BUENA AV.,"PIEDRA BUENA AV. y FERNANDEZ DE LA CRUZ, F., G...",8,Point (98896.78238426 93532.43437792),-58.475340,-34.687570,MOTO-AUTO,AUTO,04:00:00
1,2016-01-02,2016,1,2,CONDUCTOR,AUTO,MASCULINO,70,2016-01-02,1,...,GRAL PAZ,"PAZ, GRAL. AV.","PAZ, GRAL. AV. y DE LOS CORRALES AV.",9,Point (95832.05571093 95505.41641999),-58.508775,-34.669777,AUTO-PASAJEROS,PASAJEROS,01:15:00
2,2016-01-03,2016,1,3,CONDUCTOR,MOTO,MASCULINO,30,2016-01-03,1,...,AVENIDA,ENTRE RIOS AV.,ENTRE RIOS AV. 2034,1,Point (106684.29090040 99706.57687843),-58.390403,-34.631894,MOTO-AUTO,AUTO,07:00:00
3,2016-01-10,2016,1,10,CONDUCTOR,MOTO,MASCULINO,18,NaT,1,...,AVENIDA,LARRAZABAL AV.,"LARRAZABAL AV. y VILLEGAS, CONRADO, GRAL.",8,Point (99840.65224780 94269.16534422),-58.465039,-34.680930,MOTO-SD,SD,00:00:00
4,2016-01-21,2016,1,21,CONDUCTOR,MOTO,MASCULINO,29,2016-02-01,1,...,AVENIDA,SAN JUAN AV.,"SAN JUAN AV. y SAENZ PEÃ‘A, LUIS, PRES.",1,Point (106980.32827929 100752.16915795),-58.387183,-34.622466,MOTO-PASAJEROS,PASAJEROS,05:20:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
712,2021-12-12,2021,12,12,PEATON,PEATON,FEMENINO,50,2021-12-12,1,...,AVENIDA,RIVADAVIA AV.,RIVADAVIA AV. y PUEYRREDON AV.,3,Point (105258.35368554 102122.93231400),-58.405969,-34.610120,PEATON-AUTO,AUTO,06:20:00
713,2021-12-13,2021,12,13,PASAJERO_ACOMPAÑANTE,MOTO,FEMENINO,18,2021-12-18,1,...,AVENIDA,RIESTRA AV.,RIESTRA AV. y MOM,7,Point (102728.60090138 98186.24929177),-58.433538,-34.645616,MOTO-AUTO,AUTO,17:10:00
714,2021-12-20,2021,12,20,PASAJERO_ACOMPAÑANTE,MOTO,FEMENINO,43,2021-12-20,1,...,AUTOPISTA,"DELLEPIANE, LUIS, TTE. GRAL.","DELLEPIANE, LUIS, TTE. GRAL. y LACARRA AV.",9,Point (99624.29795829 97569.69801131),-58.467398,-34.651178,MOTO-AUTO,AUTO,01:10:00
715,2021-12-30,2021,12,30,CONDUCTOR,MOTO,MASCULINO,27,2022-01-02,1,...,AVENIDA,GAONA AV.,GAONA AV. y TERRADA,11,Point (99116.45492358 101045.23284826),-58.472934,-34.619847,MOTO-CARGAS,CARGAS,00:43:00


Rename the columns

In [143]:
# Se coloca la primera en mayúscula
tabla_final.columns = [x.capitalize() for x in tabla_final.columns]
# Se reemplazan los guiones por espacios
tabla_final.columns = tabla_final.columns.str.replace('_', ' ')
# Se renombran algunas columnas
tabla_final = tabla_final.rename(columns={'Fecha x': 'Fecha',
                                                      'Aaaa x':'Año',
                                                      'Mm x':'Mes',
                                                      'Dd x':'Día',
                                                      'Victima x':'Víctima'})

In [144]:
tabla_final.columns.tolist()

['Fecha',
 'Año',
 'Mes',
 'Día',
 'Rol',
 'Víctima',
 'Sexo',
 'Edad',
 'Fecha fallecimiento',
 'N victimas',
 'Hh',
 'Lugar del hecho',
 'Tipo de calle',
 'Calle',
 'Dirección normalizada',
 'Comuna',
 'Xy (caba)',
 'Pos x',
 'Pos y',
 'Participantes',
 'Acusado',
 'Hora']

In [145]:
#Save dataset for EDA
tabla_final= tabla_final.to_csv("data/total_homicidios.csv", index=False)